In [1]:
import os
from dotenv import load_dotenv
from nltk import toolbox
load_dotenv()
from IPython.display import Markdown, display

In [2]:
from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

/Users/nithurshen/SNU/MAT496/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nithurshen/SNU/MAT496/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

@tool
def ddgs(source, query):
    """Integrates with Duck Duck Go Search engine, and fetches latest information from the web
    Allowed parameters for source: 'news', 'article', 'text', 'images', 'videos', 'answers'"""
    duck = DuckDuckGoSearchRun()
    if source == 'article':
        full_query = f"{query} article"
    elif source == 'news':
        full_query = f"{query} news"
    else:
        full_query = query

    return duck.invoke(full_query)

tools = [ddgs]

model_op = model.bind_tools(tools)

In [35]:
response = model_op.invoke("Search the web about Max Verstappen' mom, and the information must be based on internet articles")
response.tool_calls

[{'name': 'ddgs',
  'args': {'query': 'Max Verstappen mom', 'source': 'article'},
  'id': 'wffzj3qdm',
  'type': 'tool_call'}]

In [36]:
tool_args = response.tool_calls[0]['args']
result = ddgs.invoke(tool_args)
print(result, response.tool_calls)

3 Dec 2024 — She is the former wife of Dutch Formula One driver Jos Verstappen, and mother to four-time World Drivers' Champion Max Verstappen . feature-image. Max Verstappen is having the best Formula One season in his career so far. The man in form has so far won five races out of the nine races in the 2021 season. Max is currently leading the drivers’ championship with 182 points, being the first Dutch driver to do so. Max Verstappen , who started karting at four, benefited from his mother’s expertise. Sophie stressed discipline, attention, and hard effort, which would shape Max ’s racing style. Max won many junior formula titles under her guidance before making his F1 debut at 17. Max Verstappen ’s mother, Sophie Kumpen, has recently hit the headlines with her controversial remark about Sergio Perez. This came after Verstappen came under massive criticism for not letting the Mexican pass during the final laps of the Brazil GP. Max Verstappen and Sergio Perez of Red Bull racing at t

In [41]:
## Creating one unified function for web search with LLM:
def web_search(prompt):
    response = model_op.invoke(prompt)
    if response.tool_calls:
        print(response.tool_calls[0]['args'])
        return ddgs.invoke(response.tool_calls[0]['args'])
    else:
        return response.content


In [42]:
web_search("Tell me about Oscar Piastri's early life, based on latest reported news!")

{'query': 'Oscar Piastri early life', 'source': 'news'}


"Outside of Formula One, Oscar Piastri leads an active and well-rounded life . ... Piastri s multifaceted interests and dedication to maintaining a ... The physical and mental tuning helping Piastri acclimatise to F1 - https://www.motorsport.com/f1/ news /th... ... Oscar Piastri , Piastri F1, McLaren F1 ... From his early karting days in Australia to his controversial Formula One debut with McLaren, Oscar Piastri s story is one of methodical brilliance ... From his early karting days in Australia to his controversial Formula One debut with McLaren, Oscar Piastri 's story is one of methodical brilliance ... Stay up-to-date on the latest news and updates about Oscar Piastri ’ s F1 career, family background, and personal life ."